# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [46]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [47]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [48]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [49]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [50]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [51]:
split_documents[5:10]

['ything else you can think of that\nwould make the world a better place. Think it should be easier\nfor low-income people to borrow money? Start Prosper. Think\ntelevision should be opened up to an inXnite number of channels? Start Joost. Think that computers should be based on Unix\nand open standards and not proprietary technology? Start Sun.\nThe ability to create your ideal culture and work with a dream team\nof people you get to assemble yourself. Want your culture to be\nbased on people who have fun every day and enjoy working\ntogether? Or, are hyper-competitive both in work and play? Or,\nare super-focused on creating innovative new rocket science\nPart 1: Why not to do a startup 3\ntechnologies? Or, are global in perspective from day one? You\nget to choose, and to build your culture and team to suit.\nAnd Xnally, money — startups done right can of course be highly\nlucrative. This is not just an issue of personal greed — when\nthings go right, your team and employees will th

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [52]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

### Answers

1. As it's stated in the documentation -> yes, we can modify the number of dimension for the text-embedding-3 and later models

2. As it's stated in the "Reducing embedding" dimensions block -> one of the tecniques is to remove some numbers from the end of the sequence. From what I understand it's possible because open ai made text-embedding-3 the way that the most valuable information is stored at the begining of the vector.


### Notes for myself

1. What's considered as most valuable information? This seems subjective. How can we control it?
2. Lowering dimensions give us decrease of volume of the vector db -> faster retrieval -> less latency for rag applications / less $
3. Usually people start to optimize the dimensions if the number of docs is at least >100K




We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [53]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

Processed batch 1/1 (373 texts)


#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

### Answer

`async` approach means that we can run opertions in parallel. `sync` means that we run operations step by step. 

hence, the pros of the `async` are:

1. speed -> it just works faster
2. effective use of the I/O time -> when we call open ai api we spend quite a lot of time waiting for response rathen than calculations -> so instead of wating we work with other tasks instead of just waiting


So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [54]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Answer

While keeping the same prompt is obvious :). According to the link we can: set temperature to 0 -> it reduces randomness in token sampling.


Also, it looks like there is a seed parameter. https://platform.openai.com/docs/api-reference/chat/create#chat-create-seed

it should work similar to other statistics python libraries, but it's in the beta now. 






### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [55]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [56]:
print(response)

The best way to write a loop in Python largely depends on what you want to achieve. However, here are some general tips and common practices to help you write effective and readable loops:

1. **Choose the Right Loop Type**: 
   - Use a `for` loop when you need to iterate over a sequence (like a list, tuple, string, or range).
   - Use a `while` loop when you want to repeat an action as long as a certain condition is true.

2. **Use Meaningful Variable Names**:
   - Choose descriptive names for your loop variables to make your code easier to read and understand.

3. **Keep the Loop Body Simple**:
   - Try to keep the logic inside the loop straightforward and concise. If the loop body becomes complex, consider moving some of the logic into a separate function.

4. **Use Built-in Functions**:
   - Take advantage of Python's built-in functions and iterators, such as `enumerate()`, `zip()`, and list comprehensions, for more concise and efficient code.

5. **Avoid Unnecessary Computations**

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [57]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [58]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [59]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: The 'Michael Eisner Memorial Weak Executive Problem' refers to the tendency of CEOs or startup founders to hire weaker executives in the functions where they themselves excel. This happens when a CEO, having come from a specific area such as product management, sales, or marketing, chooses to hire a less capable individual for that role. The underlying issue is that the CEO has difficulty letting go of the function that initially brought them success, leading to hiring someone weak so they can maintain control and continue to be perceived as the strong figure in that area. This phenomenon is illustrated by the example of Michael Eisner, the former CEO of Disney, who, despite his capabilities in television, oversaw a situation where ABC fell to fourth place under his management, highlighting the consequences of such hiring practices.

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

### Answer

1. Chain-of-Thought Prompting will do the job. It makes the model to slow down, reason more. We spend more tokens and it makes the quality of the answer better. 

2. System role prompting

3. Few-shot prompting -> giving the model some examples of good and bad output

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [60]:
import PyPDF2

class PDFFileLoader:
    def __init__(self, path: str):
        self.path = path
        self.documents = []

    def load_file(self):
        """Extract text from PDF file"""
        with open(self.path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
            self.documents.append(text)

    def load_documents(self):
        """Load and return documents"""
        if not self.documents:
            self.load_file()
        return self.documents

In [61]:
# Test PDFFileLoader with Paul Graham Essays PDF
pdf_loader = PDFFileLoader("data/paulgraham-essays.pdf")
pdf_documents = pdf_loader.load_documents()


print("\n First 200 characters:")
print(pdf_documents[0][:200])
print("\n Sample from middle:")
print(pdf_documents[0][len(pdf_documents[0])//2:len(pdf_documents[0])//2 + 200])


 First 200 characters:
How to Do Great Work - # 217
July 2023
If you collected lists of techniques for doing great work in a lot of different fields, what would the 
intersection look like? I decided to find out by making i

 Sample from middle:
ing differently as a result. [7]
Surely I'm not claiming that ideas have to have practical applications to be interesting? No, they 
may not have to. Hardy's boast that number theory had no use whatso


In [62]:
from aimakerspace.text_utils import CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase


async def create_vector_db_from_pdf(pdf_path: str):
    """
    Create vector database from a PDF file - same approach as notebook
    """
    # Load PDF
    pdf_loader = PDFFileLoader(pdf_path)
    documents = pdf_loader.load_documents()
    print(f"Loaded PDF: {pdf_path}")
    
    # Split into chunks
    text_splitter = CharacterTextSplitter()
    split_documents = text_splitter.split_texts(documents)
    print(f"Split into {len(split_documents)} chunks")
    
    # Create vector database
    vector_db = VectorDatabase()
    vector_db = await vector_db.abuild_from_list(split_documents)
    print("Vector database ready!")
    
    return vector_db

pg_vector_db = await create_vector_db_from_pdf("data/paulgraham-essays.pdf")

Loaded PDF: data/paulgraham-essays.pdf
Split into 3736 chunks
Processed batch 1/8 (512 texts)
Processed batch 2/8 (512 texts)
Processed batch 3/8 (512 texts)
Processed batch 4/8 (512 texts)
Processed batch 5/8 (512 texts)
Processed batch 6/8 (512 texts)
Processed batch 7/8 (512 texts)
Processed batch 8/8 (152 texts)
Vector database ready!


In [63]:
pg_vector_db.search_by_text("How to develop a habbit", k=3)

[("or long. Eventually something would come up that required \nme to use it more than that. And then I'd gradually slip back into my old ways.\nAddictive things have to be treated as if they were sentient adversaries—as if there were a little \nman in your head always cooking up the most plausible arguments for doing whatever you're \ntrying to stop doing. If you leave a path to it, he'll find it.\nThe key seems to be visibility. The biggest ingredient in most bad habits is denial. So you have \nto make it so that you can't merely slip into doing the thing you're trying to avoid. It has to set off \nalarms.\nMaybe in the long term the right answer for dealing with Internet distractions will be software \nthat watches and controls them. But in the meantime I've found a more drastic solution that \ndefinitely works: to set up a separate computer for using the Internet.\nI now leave wifi turned off on my main computer except when I need to transfer a file or edit a \nweb page, and I have 

In [64]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [65]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=pg_vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "How to be happy?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: According to the context provided, true happiness often stems from engagement and excitement in what one does, rather than from chasing financial security or superficial markers of success. For instance, Source 1 mentions that people who enjoy working creatively feel as if they are in their natural habitat, which makes them feel "awake and alive," even if they experience anxiety or struggle in their work. 

Source 2 suggests that the happiest people are those who love what they do, indicating that finding work you enjoy is essential, although it can be a difficult journey. It emphasizes that having a clear destination—knowing what work you love—can help in achieving happiness.

Moreover, Source 3 advocates for taking action and creating things, suggesting that if someone is unsure of what to do, they should focus on making something. The act of building and creating can lead to satisfaction and potentially happiness, steering away from excessive worrying.

In summary, happine

Different Distance Metrics for RAG
--------------------------------------------------------------

Distance Metric        | Pros (Simple)                                      | Cons (Simple)
-----------------------|----------------------------------------------------|--------------------------------------------------------------
Cosine Similarity      | Matches meaning despite sentence length; fast      | Needs normalization; ignores word count and vector size
Dot Product            | Uses direction and size; GPU-friendly              | Long texts can dominate; needs scaling
Euclidean (L2)         | Intuitive distance; works well for many tasks      | High dimensions distort scale; needs feature scaling
Manhattan (L1)         | Robust to outliers; simple math                    | Slower at scale; rarely supported in vector DBs
Jaccard Similarity     | Great for sets and keywords                        | Not for dense vectors; ignores semantic meaning
Hamming Distance       | Very fast on binary data                           | Requires hashing; loses detailed meaning
Learned / Mahalanobis  | Adapts to your data; can be most accurate          | Needs training and data; hard to explain or debug

--------------------------------------------------------------

In [66]:
# Let's add new distance metric to the RAG pipeline

import numpy as np

# 1. Define ALL distance metrics
def cosine_similarity(vector_a: np.array, vector_b: np.array) -> float:
    """Computes the cosine similarity between two vectors."""
    dot_product = np.dot(vector_a, vector_b)
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    return dot_product / (norm_a * norm_b)

def dot_product(vector_a: np.array, vector_b: np.array) -> float:
    """Computes the dot product between two vectors."""
    return np.dot(vector_a, vector_b)

def euclidean_distance(vector_a: np.array, vector_b: np.array) -> float:
    """Computes euclidean (L2) distance."""
    return -np.linalg.norm(vector_a - vector_b)  # Negative so higher scores = more similar

def manhattan_distance(vector_a: np.array, vector_b: np.array) -> float:
    """Computes manhattan (L1) distance."""
    return -np.sum(np.abs(vector_a - vector_b))  # Negative so higher scores = more similar

def jaccard_similarity(vector_a: np.array, vector_b: np.array) -> float:
    """Computes Jaccard similarity for continuous vectors."""
    threshold_a = np.mean(vector_a)
    threshold_b = np.mean(vector_b)
    
    binary_a = (vector_a > threshold_a).astype(int)
    binary_b = (vector_b > threshold_b).astype(int)
    
    intersection = np.sum(binary_a & binary_b)
    union = np.sum(binary_a | binary_b)
    
    return intersection / union if union > 0 else 0.0

def hamming_distance(vector_a: np.array, vector_b: np.array) -> float:
    """Computes Hamming distance for continuous vectors."""
    threshold_a = np.mean(vector_a)
    threshold_b = np.mean(vector_b)
    
    binary_a = (vector_a > threshold_a).astype(int)
    binary_b = (vector_b > threshold_b).astype(int)
    
    hamming_dist = np.sum(binary_a != binary_b)
    return -(hamming_dist / len(vector_a))

# 2. Distance metrics registry
DISTANCE_METRICS = {
    'cosine': cosine_similarity,
    'dot_product': dot_product,
    'euclidean': euclidean_distance,
    'manhattan': manhattan_distance,
    'jaccard': jaccard_similarity,
    'hamming': hamming_distance,
}

# 3. RAG Pipeline with switchable distance metric
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm, vector_db_retriever, 
                 response_style: str = "detailed", 
                 include_scores: bool = False,
                 distance_metric: str = "cosine"):
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores
        self.distance_metric = distance_metric
    
    def set_distance_metric(self, metric_name: str):
        """Change the distance metric dynamically"""
        if metric_name in DISTANCE_METRICS:
            self.distance_metric = metric_name
            print(f"✅ Distance metric changed to: {metric_name}")
        else:
            print(f"❌ Unknown metric: {metric_name}")
            print(f"Available: {list(DISTANCE_METRICS.keys())}")
        
    def run_pipeline(self, user_query: str, k: int = 4, distance_metric: str = None, **system_kwargs) -> dict:
        current_metric = distance_metric or self.distance_metric
        distance_func = DISTANCE_METRICS.get(current_metric, cosine_similarity)
        
        context_list = self.vector_db_retriever.search_by_text(
            user_query, 
            k=k, 
            distance_measure=distance_func
        )
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores ({current_metric}): {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "distance_metric": current_metric,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [67]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=pg_vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result_eucliadean = rag_pipeline.run_pipeline("How to be happy?", k=3, distance_metric="euclidean")

print(f"Response: {result_eucliadean['response']}")
print(f"\nContext Count: {result_eucliadean['context_count']}")
print(f"Similarity Scores: {result_eucliadean['similarity_scores']}")


result_cosine = rag_pipeline.run_pipeline("How to be happy?", k=3, distance_metric="cosine")

print(f"\n\nResponse: {result_cosine['response']}")
print(f"\nContext Count: {result_cosine['context_count']}")
print(f"Similarity Scores: {result_cosine['similarity_scores']}")

Response: Based on the provided context, happiness can be associated with engaging in work that you love and finding excitement in the process of creation. The happiest people are not necessarily those who possess financial security but those who genuinely enjoy their work (Source 2). It is important to focus on what you love and find meaning in your endeavors, even if that means navigating struggles along the way.

Furthermore, for those uncertain about their path, taking action and creating something can lead to fulfillment. For example, the suggestion in Source 3 to "make something" rather than spending too much time worrying about what to do can be a way to cultivate happiness. Engaging in projects and building things contributes to feelings of being "awake and alive," as stated in Source 1.

Overall, the journey towards happiness involves pursuing work that resonates with you, embracing challenges, and taking creative actions that can lead to a sense of fulfillment and excitement.

In [68]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=pg_vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True,
    distance_metric="hamming_distance"
)

result = rag_pipeline.run_pipeline(
    "How to be happy?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: Based on the provided context, happiness is not necessarily derived from financial security but rather from engaging in work that you love. Source 2 states, "the happiest people are not those who have it [financial security], but those who like what they do." This suggests that finding work you enjoy is paramount to happiness.

Additionally, Source 1 emphasizes that feeling excited or engaged in the process of problem-solving and creative work can lead to moments of happiness, even if those moments are fleeting. It describes how working on challenging tasks can make one feel "awake and alive," indicating that engagement in meaningful endeavors contributes positively to one's happiness.

Source 3 offers the perspective that instead of spending time worrying about what to do, one should focus on creating and building things. The idea is that taking action, whether through creativity or hands-on projects, reduces worry and enhances fulfillment.

In summary, to foster happiness a

In [69]:
# Test with metrics that show big differences
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=pg_vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

# Compare euclidean vs jaccard - very different!
result1 = rag_pipeline.run_pipeline("How to be happy?", k=3, distance_metric="euclidean")
result2 = rag_pipeline.run_pipeline("How to be happy?", k=3, distance_metric="jaccard")

print("Euclidean:", result1['similarity_scores'])
print("Jaccard  :", result2['similarity_scores'])

Euclidean: ['Source 1: -1.076', 'Source 2: -1.089', 'Source 3: -1.093']
Jaccard  : ['Source 1: 0.479', 'Source 2: 0.465', 'Source 3: 0.458']


In [70]:
## Add Metadata support


from aimakerspace.vectordatabase import VectorDatabase as BaseVectorDatabase
from aimakerspace.openai_utils.embedding import EmbeddingModel
import numpy as np
from collections import defaultdict
from typing import List, Tuple, Callable, Dict, Any

class VectorDatabase(BaseVectorDatabase):
    def __init__(self, embedding_model: EmbeddingModel = None):
        super().__init__(embedding_model)
        self.metadata = defaultdict(dict)  # Add metadata storage
    
    def insert(self, key: str, vector: np.array, metadata: Dict[str, Any] = None) -> None:
        """Enhanced insert with optional metadata"""
        super().insert(key, vector)
        self.metadata[key] = metadata or {}
    
    def get_metadata(self, key: str) -> Dict[str, Any]:
        """Get metadata for a specific document"""
        return self.metadata.get(key, {})
    
    def search(
        self,
        query_vector: np.array,
        k: int,
        distance_measure: Callable = cosine_similarity,
        metadata_filter: Dict[str, Any] = None,
    ) -> List[Tuple[str, float]]:
        """Enhanced search with optional metadata filtering"""
        
        scores = []
        for key, vector in self.vectors.items():
            if metadata_filter and not self._matches_filter(self.metadata[key], metadata_filter):
                continue
            
            score = distance_measure(query_vector, vector)
            scores.append((key, score))
        
        return sorted(scores, key=lambda x: x[1], reverse=True)[:k]
    
    def search_by_text(
        self,
        query_text: str,
        k: int,
        distance_measure: Callable = cosine_similarity,
        metadata_filter: Dict[str, Any] = None,
        return_as_text: bool = False,
    ) -> List[Tuple[str, float]]:
        """Enhanced search_by_text with metadata filtering"""
        query_vector = self.embedding_model.get_embedding(query_text)
        results = self.search(query_vector, k, distance_measure, metadata_filter)
        return [result[0] for result in results] if return_as_text else results
    
    def _matches_filter(self, metadata: Dict[str, Any], filter_criteria: Dict[str, Any]) -> bool:
        """Check if metadata matches filter criteria"""
        for key, value in filter_criteria.items():
            if key not in metadata:
                return False
            if isinstance(value, list):
                if metadata[key] not in value:
                    return False
            elif metadata[key] != value:
                return False
        return True
    
    async def abuild_from_list(self, list_of_text: List[str], metadata_list: List[Dict[str, Any]] = None) -> "VectorDatabase":
        """Enhanced abuild_from_list with optional metadata"""
        if metadata_list is None:
            metadata_list = [{}] * len(list_of_text)
        
        if len(list_of_text) != len(metadata_list):
            raise ValueError("Text list and metadata list must have same length")
        
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding, metadata in zip(list_of_text, embeddings, metadata_list):
            self.insert(text, np.array(embedding), metadata)
        return self

In [71]:
from aimakerspace.text_utils import CharacterTextSplitter
from typing import Dict, Any

async def create_vector_db_from_pdf(pdf_path: str, document_metadata: Dict[str, Any] = None):
    """
    Create vector database from a PDF file with automatic rate limiting.
    """
    # Load PDF
    pdf_loader = PDFFileLoader(pdf_path)
    documents = pdf_loader.load_documents()
    print(f"Loaded PDF: {pdf_path}")
    
    # Split into chunks
    text_splitter = CharacterTextSplitter()
    split_documents = text_splitter.split_texts(documents)
    print(f"Split into {len(split_documents)} chunks")
    
    # Create metadata for each chunk
    base_metadata = document_metadata or {}
    chunk_metadata = []
    for i, chunk in enumerate(split_documents):
        metadata = {
            **base_metadata,
            "chunk_id": i,
            "chunk_length": len(chunk),
            "source_file": pdf_path,
            "document_type": "pdf"
        }
        chunk_metadata.append(metadata)
    
    # Create vector database
    vector_db = VectorDatabase()
    vector_db = await vector_db.abuild_from_list(split_documents, chunk_metadata)
    print("Vector database ready!")
    
    return vector_db

pg_metadata_vector_db = await create_vector_db_from_pdf(
    "data/paulgraham-essays.pdf", 
    {"author": "Paul Graham", "topic": "startups"}
)

Loaded PDF: data/paulgraham-essays.pdf
Split into 3736 chunks
Processed batch 1/8 (512 texts)
Processed batch 2/8 (512 texts)
Processed batch 3/8 (512 texts)
Processed batch 4/8 (512 texts)
Processed batch 5/8 (512 texts)
Processed batch 6/8 (512 texts)
Processed batch 7/8 (512 texts)
Processed batch 8/8 (152 texts)
Vector database ready!


In [72]:
# Only search Paul Graham's content
results = pg_metadata_vector_db.search_by_text(
    "how to start a company", 
    k=5, 
    metadata_filter={"author": "Paul Graham"}
)

# Only search PDF documents  
results = pg_metadata_vector_db.search_by_text(
    "venture capital", 
    k=3, 
    metadata_filter={"document_type": "pdf"}
)

# Multiple filters
results = pg_metadata_vector_db.search_by_text(
    "startup advice", 
    k=5, 
    metadata_filter={"author": "Paul Graham", "topic": "startups"}
)

In [73]:
pg_metadata_vector_db.search_by_text("How to develop a habbit", k=3)

[("or long. Eventually something would come up that required \nme to use it more than that. And then I'd gradually slip back into my old ways.\nAddictive things have to be treated as if they were sentient adversaries—as if there were a little \nman in your head always cooking up the most plausible arguments for doing whatever you're \ntrying to stop doing. If you leave a path to it, he'll find it.\nThe key seems to be visibility. The biggest ingredient in most bad habits is denial. So you have \nto make it so that you can't merely slip into doing the thing you're trying to avoid. It has to set off \nalarms.\nMaybe in the long term the right answer for dealing with Internet distractions will be software \nthat watches and controls them. But in the meantime I've found a more drastic solution that \ndefinitely works: to set up a separate computer for using the Internet.\nI now leave wifi turned off on my main computer except when I need to transfer a file or edit a \nweb page, and I have 

In [74]:
rag_pipeline_with_metadata = RetrievalAugmentedQAPipeline(
    vector_db_retriever=pg_metadata_vector_db,  # ← NEW database (with metadata)
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline_with_metadata.run_pipeline("How to be happy?", k=3, distance_metric="hamming_distance")
result

{'response': 'Based on the context above, happiness is not solely defined by financial security, but rather by engaging in work that you enjoy and find meaningful. Source 2 suggests that "the happiest people are not those who have [financial security], but those who like what they do." Therefore, achieving happiness involves pursuing work you love and finding joy in the process rather than just the outcomes.\n\nAdditionally, Source 1 indicates that although writing or creating can be fraught with worry and puzzlement, it can also lead to moments of happiness when things work out. Engaging deeply with work, even when it is challenging, allows individuals to feel "awake and alive," suggesting that fulfillment can come from the process itself, even if it involves struggles.\n\nMoreover, Source 3 advises that if you\'re unsure about what to do, you should focus on creating or "hacking things together." This approach emphasizes taking action and enjoying the creative process, which can lead

In [75]:
metadata_with_scores = []
for chunk_text, score in result['context']:
    metadata = pg_metadata_vector_db.get_metadata(chunk_text)
    metadata['similarity_score'] = float(score)
    metadata_with_scores.append(metadata)

print(metadata_with_scores)

[{'author': 'Paul Graham', 'topic': 'startups', 'chunk_id': 181, 'chunk_length': 1000, 'source_file': 'data/paulgraham-essays.pdf', 'document_type': 'pdf', 'similarity_score': 0.4211570454895871}, {'author': 'Paul Graham', 'topic': 'startups', 'chunk_id': 2446, 'chunk_length': 1000, 'source_file': 'data/paulgraham-essays.pdf', 'document_type': 'pdf', 'similarity_score': 0.4069066793168809}, {'author': 'Paul Graham', 'topic': 'startups', 'chunk_id': 2270, 'chunk_length': 1000, 'source_file': 'data/paulgraham-essays.pdf', 'document_type': 'pdf', 'similarity_score': 0.40284561361369664}]


In [76]:
## 

## Notes for the Loom

# Why did we do in Python?
# 


In [77]:
### YOUR CODE HERE